# 计步及运动识别 Jupyter Notebook

本笔记本实现了基于加速度计数据的计步和活动识别功能。

- **计步 (Step Counting)**: 使用峰值检测算法计算步数。
- **运动识别 (Motion Recognition)**: 使用机器学习模型识别不同活动类型,如行走、跑步、静止等。

数据来源: 加速度计原始数据,经过预处理后用于分析。

## 步骤概述
1. 导入必要的库
2. 加载和预处理传感器数据
3. 实现计步算法
4. 特征提取用于活动识别
5. 训练活动识别模型
6. 评估和可视化结果

In [18]:
import numpy as np
import pandas as pd
import csv
import os
import struct
from datetime import datetime
from zoneinfo import ZoneInfo
from dataclasses import dataclass, field
from typing import List, Optional,Tuple
from scipy import signal
from scipy.signal import resample
import matplotlib.pyplot as plt
import ipywidgets as widgets
import matplotlib.dates as mdates


import matplotlib.pyplot as plt
import seaborn as sns
# 设置matplotlib参数
plt.style.use('default')  # 使用默认样式

# 尝试设置中文字体支持(如果失败则使用默认字体)
try:
    plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False
except:
    print("注意: 中文字体设置失败,将使用默认字体")
    pass

from script.data_structures import BaseInfo, ACC_Raw_Data, WearInfo, PPG_Raw_Data, TYHX_PPG_Data
from script.data_types import get_full_data_type
from script.common import get_raw_data, parse_filename, expand_acc_data, expand_ppg_data, process_and_merge_sensor_data
from script.wear_detection_dll import *
from script.motion_recognition_dll import *

In [12]:
def process_and_merge_dataframes(df, motion_recognition_df, acc_df):
    # 填充 df,使 unix_timestamp 连续从 df 的最小值到最大值

    # 设置 unix_timestamp 为索引
    df.set_index('unix_timestamp', inplace=True)

    # 处理重复索引,保留第一个
    if df.index.has_duplicates:
        df = df[~df.index.duplicated(keep='first')]

    # 从 df 中获取 start_ts 和 end_ts
    start_ts = df.index.min()
    end_ts = df.index.max()

    # 创建完整的 timestamp 范围
    full_index = pd.Index(range(start_ts, end_ts + 1), name='unix_timestamp')

    # 使用 reindex 填充缺失的行
    df_filled = df.reindex(full_index)

    # 填充 activity_level 和 step 使用线性插值,然后转为 int
    df_filled['activity_level'] = df_filled['activity_level'].interpolate(method='linear').astype(int)
    df_filled['step'] = df_filled['step'].interpolate(method='linear').astype(int)

    # 填充 wear_flag 使用前向填充
    df_filled['wear_flag'] = df_filled['wear_flag'].ffill().astype(int)

    # 对于其他列,如果需要填充,可以添加类似代码
    # 例如,如果 heart_rate 和 step_freq 也需要填充,可以指定方法
    # 这里假设 heart_rate 使用前向填充,step_freq 使用线性插值
    df_filled['heart_rate'] = df_filled['heart_rate'].ffill().astype(int)
    df_filled['step_freq'] = df_filled['step_freq'].interpolate(method='linear').astype(int)

    # 重置索引
    df_filled.reset_index(inplace=True)

    # 为 motion_recognition_df 和 acc_df 填充连续的 unix_timestamp
    # 使用起始 unix_timestamp + 行计数

    if motion_recognition_df is not None and not motion_recognition_df.empty:
        start_ts_motion = motion_recognition_df['unix_timestamp'].min()
        motion_recognition_df['unix_timestamp'] = [start_ts_motion + i for i in range(len(motion_recognition_df))]

    if acc_df is not None and not acc_df.empty:
        start_ts_acc = acc_df['unix_timestamp'].min()
        acc_df['unix_timestamp'] = [start_ts_acc + i for i in range(len(acc_df))]

    # 分别合并 DataFrame
    # 先合并 df_filled 和 acc_df
    merged_acc_df = pd.merge(df_filled, acc_df, on='unix_timestamp', how='inner')


    # 然后合并 merged_acc_df 和 motion_recognition_df
    if motion_recognition_df is not None and not motion_recognition_df.empty:
        final_merged_df = pd.merge(merged_acc_df, motion_recognition_df, on='unix_timestamp', how='inner')
    else:
        final_merged_df = merged_acc_df



    return final_merged_df

In [3]:
def Motiondll_with_dataframe(df, directory, file_name_without_ext):
    """
    使用 DataFrame 数据调试 DLL,返回结果 DataFrame
    """
    result_list = []
    
    log_buffer = []

    # 2. 定义回调：只存列表,不写文件
    def memory_logging_callback(message):
        # print(message, end='') 
        log_buffer.append(message) # 存入内存
    
    dll_path = r"G:\ncs_workspace\application\algorithm_lib\alg_motion_recognition\build\bin\alg_motion_recognition_ring.dll"

    prev_step = None  # 用于计算步数差分
    
    try:
        with MotionRecognitionDLL(dll_path,print_callback=memory_logging_callback) as dll:
            # # 获取版本
            # version = dll.get_version()
            # print(f"DLL版本: {version.major}.{version.minor}.{version.patch}")
            
            # 初始化
            dll.init_default()

            num_calls = len(df)
            for i in range(num_calls):
                row = df.iloc[i]
                
                # 计算每秒步数(差分)
                current_step = row['step']
                if prev_step is not None:
                    steps_per_sec = current_step - prev_step
                else:
                    steps_per_sec = 0  # 第一行设为0
                
                prev_step = current_step
                
                # 提取 acc_data
                acc_data = row['acc_data']
                
                # 创建 activity_data
                activity_data = MotionActivityData()
                activity_data.unix_timestamp = int(row['unix_timestamp'])
                activity_data.wear_flag = 1 # int(row['wear_flag'])
                activity_data.heartrate = int(row['heart_rate'])
                activity_data.steps = int(steps_per_sec)  # 使用差分步数
                activity_data.step_freq = int(row['step_freq'])
                activity_data.activity_level = int(row['activity_level'])
                activity_data.sport_type = 255  # 默认值
                activity_data.sport_pause_state = 1  # 默认值
                
                # 调用 process
                result = dll.process(acc_data, activity_data)
                
                # # 打印结果结构体所有字段
                # print(f"调用 {i+1}: unix_timestamp={activity_data.unix_timestamp}")
                # print(f"  Result: error_code={result.error_code}, init_timer={result.init_timer}, motion_type={result.motion_type}, detection_state={result.detection_state}")
                # print(f"  Result: activity_motion_time={result.activity_motion_time}, auto_detect_state={result.auto_detect_state}, instant_type={result.instant_type}")
                # print(f"  Result: hr_rising={result.hr_rising}, hr_avg={result.hr_avg}, steps_per_minute={result.steps_per_minute}, acc_feature={result.acc_feature}")
                # print(f"  Result: avg_activity_level={result.avg_activity_level}, wear_flag={result.wear_flag}, unworn_seconds={result.unworn_seconds}")
                # print(f"  ActivityData: unix_timestamp={activity_data.unix_timestamp}, wear_flag={activity_data.wear_flag}, heartrate={activity_data.heartrate}")
                # print(f"  ActivityData: steps={activity_data.steps}, step_freq={activity_data.step_freq}, activity_level={activity_data.activity_level}")
                # print(f"  ActivityData: sport_type={activity_data.sport_type}, sport_pause_state={activity_data.sport_pause_state}")
                # print(f"  AccData: sensor_size={len(acc_data)}, sample_count={len(acc_data)}")
                # print()
                
                # row
                result_list.append({**row.to_dict(), **dll.result_to_dict(result)})

            # 将 result_list 转换为 Python DataFrame

            # 转换所有结果为字典列表


            # 创建 DataFrame
            df_results = pd.DataFrame(result_list)

    finally:
        log_file_path = os.path.join(directory, f"{file_name_without_ext}_log.txt")
        try:
            with open(log_file_path, 'w', encoding='utf-8') as f:
                # 将列表中的字符串拼接后写入,或者逐行写入
                # 注意：如果 message 自带换行符,就不用加 \n
                for msg in log_buffer:
                    f.write(msg) 
            print(f"日志已保存至: {log_file_path}")
        except Exception as io_e:
            print(f"保存日志失败: {io_e}")

    return df_results

In [4]:
def plot_acc_and_step_data(df,directory, file_name_without_ext):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 12)) #, sharex=True)

    # 添加文件名作为整个图的标题
    fig.suptitle(file_name_without_ext, fontsize=14, fontweight='bold', y=0.995)

    # 提取 acc 数据 - 展开所有采样点
    if 'acc_data' in df.columns:
        # 展开所有采样点到单独的列表
        all_x = []
        all_y = []
        all_z = []
        
        for acc_data in df['acc_data']:
            if isinstance(acc_data, list) and len(acc_data) > 0:
                for point in acc_data:
                    if isinstance(point, (list, tuple)) and len(point) >= 3:
                        all_x.append(point[0])
                        all_y.append(point[1])
                        all_z.append(point[2])
        
        # acc_magnitude = np.sqrt(np.array(all_x)**2 + np.array(all_y)**2 + np.array(all_z)**2)

        # 在 ax1 绘制 acc x, y, z
        ax1.plot(all_x, label='ACC X', color='red', linewidth=0.5)
        ax1.plot(all_y, label='ACC Y', color='green', linewidth=0.5)
        ax1.plot(all_z, label='ACC Z', color='blue', linewidth=0.5)
        # ax1.plot(acc_magnitude, label='ACC Magnitude', color='black', linewidth=0.5)
        ax1.set_ylabel('Acceleration')
        ax1.legend()
        totle_time = df['unix_timestamp'].iloc[-1] - df['unix_timestamp'].iloc[0]
        ax1.set_title(f'Acceleration Data {totle_time} seconds - {len(all_x)} samples')

    # 在 ax2 绘制 step 和 step 的差分
    if 'step' in df.columns:
        step_values = df['step'].values
        step_diff = np.diff(step_values, prepend=step_values[0])
        
        # 获取起始、结束和总步数
        step_start = df['step'].iloc[0]
        step_end = df['step'].iloc[-1]
        step_total = step_end - step_start

        # ax2.plot(df['step'], label='Step', color='orange')
        ax2.plot(df['unix_timestamp'] ,   step_diff, label='Step/sec', color='purple')
        ax2.set_ylabel('Step')
        ax2.legend()
        ax2.set_title(f'Step Data - start: {step_start} stop: {step_end} total: {step_total}')


    
    plt.tight_layout()
    save_fig_name = os.path.join(directory, f"{file_name_without_ext}_acc_step.png")
    plt.savefig(save_fig_name,dpi=300)
    # plt.close()
    plt.show()

In [5]:
def plot_motion_recognition_df(result, directory, file_name_without_ext, is_show=False):
    """
    绘制活动识别结果的多个指标 (适配 motion_recognition_df 数据结构)
    
    参数:
        result: DataFrame, 包含活动识别结果的数据框
        directory: str, 保存目录
        file_name_without_ext: str, 文件名(不含扩展名),用于图表标题
        is_show: bool, 是否显示图表(默认为 False, 保存到文件)
    """
    
    # 创建子图 (5x2 布局,显示更多指标)
    fig, axes = plt.subplots(5, 2, figsize=(18, 15), sharex=True)
    fig.suptitle(f'Motion Recognition Results (Ring) - {file_name_without_ext}', fontsize=16)

    # 绘制 motion_type_ring
    axes[0, 0].plot(result['unix_timestamp'], result['motion_type_ring'], label='motion_type_ring', color='blue', linewidth=1.5)
    axes[0, 0].set_title('Motion Type (Ring)')
    axes[0, 0].set_ylabel('Type')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].legend()

    # 绘制 detection_state
    axes[0, 1].plot(result['unix_timestamp'], result['detection_state'], label='detection_state', color='green', linewidth=1.5)
    axes[0, 1].set_title('Detection State')
    axes[0, 1].set_ylabel('State')
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].legend()

    # 绘制 activity_motion_time_ring
    axes[1, 0].plot(result['unix_timestamp'], result['activity_motion_time_ring'], label='activity_motion_time_ring', color='red', linewidth=1.5)
    axes[1, 0].set_title('Activity Motion Time (Ring)')
    axes[1, 0].set_ylabel('Time (s)')
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].legend()

    # 绘制 auto_detect_state
    axes[1, 1].plot(result['unix_timestamp'], result['auto_detect_state'], label='auto_detect_state', color='purple', linewidth=1.5)
    axes[1, 1].set_title('Auto Detect State')
    axes[1, 1].set_ylabel('State')
    axes[1, 1].grid(True, alpha=0.3)
    axes[1, 1].legend()

    # 绘制 instant_type
    axes[2, 0].plot(result['unix_timestamp'], result['instant_type'], label='instant_type', color='orange', linewidth=1.5)
    axes[2, 0].set_title('Instant Type')
    axes[2, 0].set_ylabel('Type')
    axes[2, 0].grid(True, alpha=0.3)
    axes[2, 0].legend()

    # 绘制 steps_per_minute
    axes[2, 1].plot(result['unix_timestamp'], result['steps_per_minute'], label='steps_per_minute', color='brown', linewidth=1.5)
    axes[2, 1].set_title('Steps Per Minute')
    axes[2, 1].set_ylabel('Steps/min')
    axes[2, 1].grid(True, alpha=0.3)
    axes[2, 1].legend()

    # 绘制 acc_feature
    axes[3, 0].plot(result['unix_timestamp'], result['acc_feature'], label='acc_feature', color='hotpink', linewidth=1.5)
    axes[3, 0].set_title('ACC Feature')
    axes[3, 0].set_ylabel('Feature Value')
    axes[3, 0].grid(True, alpha=0.3)
    axes[3, 0].legend()

    # 绘制 avg_activity_level
    axes[3, 1].plot(result['unix_timestamp'], result['avg_activity_level'], label='avg_activity_level', color='cyan', linewidth=1.5)
    axes[3, 1].set_title('Average Activity Level')
    axes[3, 1].set_ylabel('Level')
    axes[3, 1].grid(True, alpha=0.3)
    axes[3, 1].legend()

    # 绘制 hr_avg (心率平均值)
    axes[4, 0].plot(result['unix_timestamp'], result['hr_avg'], label='hr_avg', color='crimson', linewidth=1.5)
    axes[4, 0].set_title('Heart Rate Average')
    axes[4, 0].set_ylabel('HR (bpm)')
    axes[4, 0].set_xlabel('Unix Timestamp')
    axes[4, 0].grid(True, alpha=0.3)
    axes[4, 0].legend()

    # 绘制 wear_flag_ring
    axes[4, 1].plot(result['unix_timestamp'], result['wear_flag_ring'], label='wear_flag_ring', color='navy', linewidth=1.5)
    axes[4, 1].set_title('Wear Flag (Ring)')
    axes[4, 1].set_ylabel('Flag')
    axes[4, 1].set_xlabel('Unix Timestamp')
    axes[4, 1].grid(True, alpha=0.3)
    axes[4, 1].legend()

    plt.tight_layout()

    if is_show:
        plt.show()
    else:
        file_path = os.path.join(directory, f"{file_name_without_ext}_motion_recognition_ring.png")
        plt.savefig(file_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"图表已保存: {file_path}")

    # return fig

In [10]:
def analyze_motion_recognition(df):
    """
    分析 motion_recognition_df 的运动识别状态
    
    统计内容:
    1. 运行总时间
    2. 数据完整性检查(开始/结束状态)
    3. detection_state 0-1-0 循环次数和每次时间
    4. detection_state 0-1-2...7-0 完整循环次数和从2到7的时间
    5. detection_state 状态2-7时的 motion_type_ring 分布
    6. 不完整循环统计
    """
    
    print("=" * 80)
    print("运动识别数据分析报告")
    print("=" * 80)
    
    # 1. 计算总运行时间
    if len(df) > 0:
        total_time = df['unix_timestamp'].iloc[-1] - df['unix_timestamp'].iloc[0]
        print(f"\n总运行时间: {total_time} 秒 ( {total_time/60:.2f} 分钟 )")
    
    # 2. 数据完整性检查
    detection_states = df['detection_state'].values
    timestamps = df['unix_timestamp'].values
    
    start_state = detection_states[0]
    end_state = detection_states[-1]
    
    print(f"\n数据完整性检查:")
    print(f"   起始状态: detection_state = {start_state} {'(正常)' if start_state == 0 else '(异常,应为0)'}")
    print(f"   结束状态: detection_state = {end_state} {'(正常)' if end_state == 0 else '(异常,未清空)'}")
    
    # 统计是否存在不完整的循环
    incomplete_start = start_state != 0
    incomplete_end = end_state != 0
    
    # 3. 分析 detection_state 的状态转换
    
    # 检测 0->1->0 的循环
    cycles_0_1_0 = []
    i = 0
    while i < len(detection_states):
        if detection_states[i] == 0:
            # 寻找 0->1
            j = i + 1
            while j < len(detection_states) and detection_states[j] == 0:
                j += 1
            
            if j < len(detection_states) and detection_states[j] == 1:
                # 找到了 0->1,记录状态1的开始
                state_1_start_idx = j
                state_1_start_time = timestamps[j]
                
                # 寻找 1->0,找到状态1的结束
                k = j + 1
                while k < len(detection_states) and detection_states[k] == 1:
                    k += 1
                
                if k < len(detection_states) and detection_states[k] == 0:
                    # 找到完整的 0->1->0 循环
                    state_1_end_idx = k - 1  # 状态1的最后一个位置
                    state_1_end_time = timestamps[state_1_end_idx]
                    duration = state_1_end_idx - state_1_start_idx + 1  # 数据点数量
                    
                    cycles_0_1_0.append({
                        'state_1_start_idx': state_1_start_idx,
                        'state_1_end_idx': state_1_end_idx,
                        'state_1_start_time': state_1_start_time,
                        'state_1_end_time': state_1_end_time,
                        'duration': duration
                    })
                    i = k
                else:
                    i = k if k < len(detection_states) else len(detection_states)
            else:
                i = j if j < len(detection_states) else len(detection_states)
        else:
            i += 1
    
    print(f"\n检测到 0->1->0 循环: {len(cycles_0_1_0)} 次")
    if cycles_0_1_0:
        print(f"   状态 1 的详细信息:")
        for idx, cycle in enumerate(cycles_0_1_0, 1):
            print(f"      第 {idx} 次: 索引 {cycle['state_1_start_idx']}-{cycle['state_1_end_idx']}, "
                  f"时间戳 {cycle['state_1_start_time']}-{cycle['state_1_end_time']}, "
                  f"持续 {cycle['duration']} 秒")
    
    # 4. 检测 0->1->2...->7->0 的完整循环
    cycles_0_to_7 = []
    incomplete_cycles = []  # 记录不完整的循环
    i = 0
    while i < len(detection_states):
        if detection_states[i] == 0:
            # 寻找 0->1
            j = i + 1
            while j < len(detection_states) and detection_states[j] == 0:
                j += 1
            
            if j < len(detection_states) and detection_states[j] == 1:
                # 找到 0->1,记录状态1的位置
                state_1_idx = j
                
                # 寻找到达 2 的位置
                k = j + 1
                while k < len(detection_states) and detection_states[k] not in [0, 2]:
                    k += 1
                
                if k < len(detection_states) and detection_states[k] == 2:
                    # 找到了 2,记录开始时间
                    state_2_start = timestamps[k]
                    state_2_idx = k
                    
                    # 寻找到达 7 的位置
                    m = k + 1
                    found_7 = False
                    while m < len(detection_states):
                        if detection_states[m] == 7:
                            found_7 = True
                            state_7_time = timestamps[m]
                            state_7_idx = m
                            
                            # 继续寻找返回到 0
                            n = m + 1
                            while n < len(detection_states) and detection_states[n] != 0:
                                n += 1
                            
                            if n < len(detection_states):
                                # 找到完整循环,统计每个状态的时长(数据点数量)和索引范围
                                state_durations = {}
                                state_ranges = {}
                                
                                # 从状态1开始到返回0之前,统计各状态的数据点数量和索引范围
                                for state_num in range(1, 8):
                                    state_count = 0
                                    state_start = None
                                    state_end = None
                                    for idx in range(state_1_idx, n):
                                        if detection_states[idx] == state_num:
                                            if state_start is None:
                                                state_start = idx
                                            state_end = idx
                                            state_count += 1
                                    state_durations[f'state_{state_num}_duration'] = state_count
                                    state_ranges[f'state_{state_num}_range'] = (state_start, state_end) if state_start is not None else (None, None)
                                
                                # 计算状态2-7的总时长
                                duration_2_to_7 = sum([state_durations[f'state_{s}_duration'] for s in range(2, 8)])
                                
                                # 统计该循环中状态2-7各状态时的 motion_type_ring 分布
                                motion_types_in_states = {}
                                for state_num in range(2, 8):
                                    motion_types_in_states[f'state_{state_num}'] = {}
                                    idx = state_1_idx
                                    while idx < n:
                                        if detection_states[idx] == state_num:
                                            # 获取该位置的 motion_type_ring
                                            motion_type = df.iloc[idx]['motion_type_ring']
                                            
                                            # 找到该 motion_type 的连续区间
                                            start_idx = idx
                                            while idx < n and detection_states[idx] == state_num and df.iloc[idx]['motion_type_ring'] == motion_type:
                                                idx += 1
                                            end_idx = idx - 1
                                            
                                            # 计算该 motion_type 的数据点数量
                                            duration = end_idx - start_idx + 1
                                            
                                            if motion_type not in motion_types_in_states[f'state_{state_num}']:
                                                motion_types_in_states[f'state_{state_num}'][motion_type] = 0
                                            motion_types_in_states[f'state_{state_num}'][motion_type] += duration
                                        else:
                                            idx += 1
                                
                                cycles_0_to_7.append({
                                    'start_idx': i,
                                    'state_1_idx': state_1_idx,
                                    'state_2_idx': state_2_idx,
                                    'state_7_idx': state_7_idx,
                                    'end_idx': n,
                                    'state_2_time': state_2_start,
                                    'state_7_time': state_7_time,
                                    'duration_2_to_7': duration_2_to_7,
                                    'motion_types_in_states': motion_types_in_states,  # 状态2-7的motion_type分布
                                    **state_durations,  # 展开各状态时长
                                    **state_ranges  # 展开各状态索引范围
                                })
                                i = n
                            else:
                                # 到达7但没有返回0(数据结束或异常)
                                # 统计各状态的时长和范围
                                state_durations = {}
                                state_ranges = {}
                                for state_num in range(1, 8):
                                    state_count = 0
                                    state_start = None
                                    state_end = None
                                    for idx in range(state_1_idx, len(detection_states)):
                                        if detection_states[idx] == state_num:
                                            if state_start is None:
                                                state_start = idx
                                            state_end = idx
                                            state_count += 1
                                    state_durations[f'state_{state_num}_duration'] = state_count
                                    state_ranges[f'state_{state_num}_range'] = (state_start, state_end) if state_start is not None else (None, None)
                                
                                # 统计 motion_type_ring 分布
                                motion_types_in_states = {}
                                for state_num in range(2, 8):
                                    motion_types_in_states[f'state_{state_num}'] = {}
                                    idx = state_1_idx
                                    while idx < len(detection_states):
                                        if detection_states[idx] == state_num:
                                            motion_type = df.iloc[idx]['motion_type_ring']
                                            start_idx = idx
                                            while idx < len(detection_states) and detection_states[idx] == state_num and df.iloc[idx]['motion_type_ring'] == motion_type:
                                                idx += 1
                                            end_idx = idx - 1
                                            duration = end_idx - start_idx + 1
                                            if motion_type not in motion_types_in_states[f'state_{state_num}']:
                                                motion_types_in_states[f'state_{state_num}'][motion_type] = 0
                                            motion_types_in_states[f'state_{state_num}'][motion_type] += duration
                                        else:
                                            idx += 1
                                
                                incomplete_cycles.append({
                                    'type': '到达状态7但未返回0',
                                    'start_idx': i,
                                    'state_1_idx': state_1_idx,
                                    'state_7_idx': state_7_idx,
                                    'end_idx': len(detection_states) - 1,
                                    'final_state': detection_states[-1],
                                    'motion_types_in_states': motion_types_in_states,
                                    **state_durations,
                                    **state_ranges
                                })
                                i = n
                            break
                        elif detection_states[m] == 0:
                            # 没到7就返回0了(不完整的循环)
                            # 统计各状态的时长和范围
                            state_durations = {}
                            state_ranges = {}
                            for state_num in range(1, 8):
                                state_count = 0
                                state_start = None
                                state_end = None
                                for idx in range(state_1_idx, m):
                                    if detection_states[idx] == state_num:
                                        if state_start is None:
                                            state_start = idx
                                        state_end = idx
                                        state_count += 1
                                state_durations[f'state_{state_num}_duration'] = state_count
                                state_ranges[f'state_{state_num}_range'] = (state_start, state_end) if state_start is not None else (None, None)
                            
                            # 统计 motion_type_ring 分布
                            motion_types_in_states = {}
                            for state_num in range(2, 8):
                                motion_types_in_states[f'state_{state_num}'] = {}
                                idx = state_1_idx
                                while idx < m:
                                    if detection_states[idx] == state_num:
                                        motion_type = df.iloc[idx]['motion_type_ring']
                                        start_idx = idx
                                        while idx < m and detection_states[idx] == state_num and df.iloc[idx]['motion_type_ring'] == motion_type:
                                            idx += 1
                                        end_idx = idx - 1
                                        duration = end_idx - start_idx + 1
                                        if motion_type not in motion_types_in_states[f'state_{state_num}']:
                                            motion_types_in_states[f'state_{state_num}'][motion_type] = 0
                                        motion_types_in_states[f'state_{state_num}'][motion_type] += duration
                                    else:
                                        idx += 1
                            
                            incomplete_cycles.append({
                                'type': '未到达状态7就返回0',
                                'start_idx': i,
                                'state_1_idx': state_1_idx,
                                'state_2_idx': state_2_idx,
                                'end_idx': m,
                                'max_state': max(detection_states[state_2_idx:m]),
                                'motion_types_in_states': motion_types_in_states,
                                **state_durations,
                                **state_ranges
                            })
                            break
                        else:
                            m += 1
                    
                    if not found_7 and m >= len(detection_states):
                        # 到达数据末尾但没有找到7(不完整的循环)
                        # 统计各状态的时长和范围
                        state_durations = {}
                        state_ranges = {}
                        for state_num in range(1, 8):
                            state_count = 0
                            state_start = None
                            state_end = None
                            for idx in range(state_1_idx, len(detection_states)):
                                if detection_states[idx] == state_num:
                                    if state_start is None:
                                        state_start = idx
                                    state_end = idx
                                    state_count += 1
                            state_durations[f'state_{state_num}_duration'] = state_count
                            state_ranges[f'state_{state_num}_range'] = (state_start, state_end) if state_start is not None else (None, None)
                        
                        # 统计 motion_type_ring 分布
                        motion_types_in_states = {}
                        for state_num in range(2, 8):
                            motion_types_in_states[f'state_{state_num}'] = {}
                            idx = state_1_idx
                            while idx < len(detection_states):
                                if detection_states[idx] == state_num:
                                    motion_type = df.iloc[idx]['motion_type_ring']
                                    start_idx = idx
                                    while idx < len(detection_states) and detection_states[idx] == state_num and df.iloc[idx]['motion_type_ring'] == motion_type:
                                        idx += 1
                                    end_idx = idx - 1
                                    duration = end_idx - start_idx + 1
                                    if motion_type not in motion_types_in_states[f'state_{state_num}']:
                                        motion_types_in_states[f'state_{state_num}'][motion_type] = 0
                                    motion_types_in_states[f'state_{state_num}'][motion_type] += duration
                                else:
                                    idx += 1
                        
                        incomplete_cycles.append({
                            'type': '数据结束前未到达状态7',
                            'start_idx': i,
                            'state_1_idx': state_1_idx,
                            'state_2_idx': state_2_idx,
                            'end_idx': len(detection_states) - 1,
                            'max_state': max(detection_states[state_2_idx:]),
                            'motion_types_in_states': motion_types_in_states,
                            **state_durations,
                            **state_ranges
                        })
                        i = m if m < len(detection_states) else len(detection_states)
                else:
                    i = k if k < len(detection_states) else len(detection_states)
            else:
                i = j if j < len(detection_states) else len(detection_states)
        else:
            i += 1
    
    print(f"\n检测到 0->1->2...->7->0 完整循环: {len(cycles_0_to_7)} 次")
    if cycles_0_to_7:
        for idx, cycle in enumerate(cycles_0_to_7, 1):
            total_duration = sum([cycle[f'state_{s}_duration'] for s in range(1, 8)])
            print(f"   第 {idx} 次: 总时长 {total_duration} 秒 ( {cycle['state_1_idx']}-{cycle['end_idx']-1} )")
            
            # 打印每个状态的时长和索引范围
            for state_num in range(1, 8):
                duration = cycle[f'state_{state_num}_duration']
                state_range = cycle[f'state_{state_num}_range']
                if duration > 0 and state_range[0] is not None:
                    print(f"         状态 {state_num}: {duration} 秒 ( {state_range[0]}-{state_range[1]} )")
                else:
                    print(f"         状态 {state_num}: {duration} 秒")
            
            # 打印该循环中状态2-7的 motion_type_ring 分布
            if cycle['motion_types_in_states']:
                for state_num in range(2, 8):
                    state_key = f'state_{state_num}'
                    if cycle['motion_types_in_states'][state_key]:
                        print(f"         状态 {state_num} 时的motion_type_ring分布:")
                        for motion_type, duration in sorted(cycle['motion_types_in_states'][state_key].items()):
                            print(f"            motion_type = {motion_type}: {duration} 秒")
    
    # 5. 统计不完整循环
    print(f"\n不完整循环统计: {len(incomplete_cycles)} 次")
    if incomplete_cycles:
        for idx, cycle in enumerate(incomplete_cycles, 1):
            print(f"   第 {idx} 次: {cycle['type']}")
            
            # 计算总时长
            total_duration = sum([cycle.get(f'state_{s}_duration', 0) for s in range(1, 8)])
            print(f"         总时长 {total_duration} 秒 ( {cycle['start_idx']}-{cycle['end_idx']} )")
            
            # 打印每个状态的时长和索引范围
            for state_num in range(1, 8):
                duration = cycle.get(f'state_{state_num}_duration', 0)
                state_range = cycle.get(f'state_{state_num}_range', (None, None))
                if duration > 0 and state_range[0] is not None:
                    print(f"         状态 {state_num}: {duration} 秒 ( {state_range[0]}-{state_range[1]} )")
            
            if 'max_state' in cycle:
                print(f"         最大状态: {cycle['max_state']}")
            if 'final_state' in cycle:
                print(f"         最终状态: {cycle['final_state']}")
            
            # 打印 motion_type_ring 分布
            if 'motion_types_in_states' in cycle and cycle['motion_types_in_states']:
                for state_num in range(2, 8):
                    state_key = f'state_{state_num}'
                    if state_key in cycle['motion_types_in_states'] and cycle['motion_types_in_states'][state_key]:
                        print(f"         状态 {state_num} 时的motion_type_ring分布:")
                        for motion_type, duration in sorted(cycle['motion_types_in_states'][state_key].items()):
                            print(f"            motion_type = {motion_type}: {duration} 秒")
    
    # 6. 检查开始时的不完整状态
    if incomplete_start:
        print(f"\n数据开始时已处于状态 {start_state} ,可能存在不完整的前置循环")
        # 找到第一次回到0的位置
        first_zero_idx = None
        for i in range(len(detection_states)):
            if detection_states[i] == 0:
                first_zero_idx = i
                break
        
        if first_zero_idx is not None and first_zero_idx > 0:
            # 统计各状态的时长和范围
            state_durations_start = {}
            state_ranges_start = {}
            for state_num in range(1, 8):
                state_count = 0
                state_start = None
                state_end = None
                for idx in range(0, first_zero_idx + 1):
                    if detection_states[idx] == state_num:
                        if state_start is None:
                            state_start = idx
                        state_end = idx
                        state_count += 1
                state_durations_start[state_num] = state_count
                state_ranges_start[state_num] = (state_start, state_end) if state_start is not None else (None, None)
            
            total_duration_start = sum(state_durations_start.values())
            print(f"   总时长 {total_duration_start} 秒 ( 0-{first_zero_idx} )")
            
            for state_num in range(1, 8):
                duration = state_durations_start[state_num]
                state_range = state_ranges_start[state_num]
                if duration > 0 and state_range[0] is not None:
                    print(f"         状态 {state_num}: {duration} 秒 ( {state_range[0]}-{state_range[1]} )")
            
            max_state_start = max(detection_states[:first_zero_idx+1])
            print(f"   该段最大状态: {max_state_start}")
    
    # 7. 检查结束时的不完整状态
    if incomplete_end:
        print(f"\n数据结束时处于状态 {end_state} ,循环未完成")
        # 找到最后一次为0的位置
        last_zero_idx = None
        for i in range(len(detection_states) - 1, -1, -1):
            if detection_states[i] == 0:
                last_zero_idx = i
                break
        
        if last_zero_idx is not None and last_zero_idx < len(detection_states) - 1:
            # 统计各状态的时长和范围
            state_durations_end = {}
            state_ranges_end = {}
            for state_num in range(1, 8):
                state_count = 0
                state_start = None
                state_end = None
                for idx in range(last_zero_idx, len(detection_states)):
                    if detection_states[idx] == state_num:
                        if state_start is None:
                            state_start = idx
                        state_end = idx
                        state_count += 1
                state_durations_end[state_num] = state_count
                state_ranges_end[state_num] = (state_start, state_end) if state_start is not None else (None, None)
            
            total_duration_end = sum(state_durations_end.values())
            print(f"   总时长 {total_duration_end} 秒 ( {last_zero_idx}-{len(detection_states)-1} )")
            
            for state_num in range(1, 8):
                duration = state_durations_end[state_num]
                state_range = state_ranges_end[state_num]
                if duration > 0 and state_range[0] is not None:
                    print(f"         状态 {state_num}: {duration} 秒 ( {state_range[0]}-{state_range[1]} )")
            
            max_state_end = max(detection_states[last_zero_idx:])
            print(f"   该段最大状态: {max_state_end}")
    
    print("\n" + "=" * 80)
    
    return {
        'total_time': total_time if len(df) > 0 else 0, # type: ignore
        'start_state': start_state,
        'end_state': end_state,
        'incomplete_start': incomplete_start,
        'incomplete_end': incomplete_end,
        'cycles_0_1_0': cycles_0_1_0,
        'cycles_0_to_7': cycles_0_to_7,
        'incomplete_cycles': incomplete_cycles
    }

In [9]:

directory = r'rawdata/Motion/0108/'

all_files = os.listdir(directory)

# 过滤出 .rawdata 文件
rawdata_files = [f for f in all_files if f.endswith('.rawdata')]
rawdata_files

['HeartRate_Static_lk_20260108_154931_c460.rawdata',
 'HeartRate_Static_pjy_20260108_162240_75c3.rawdata',
 'HeartRate_Static_zhangz_20260108_110047_c460.rawdata',
 'MotionRecognition_allDayAcquisition_lhq_20260108_153110_75c3.rawdata',
 'Sleep_Nap_zhangz_20260108_165124_ead7.rawdata']

In [16]:
%matplotlib qt 
# %matplotlib inline



# Wear_Wearing_shb_20260107_183543_c460  吃饭
# Wear_Wearing_shb_20260107_192811_c460  户外骑行
# Wear_Wearing_shb_20260107_193820_c460  办公


#  'HeartRate_Static_lk_20260108_154931_c460.rawdata',    
#  'HeartRate_Static_pjy_20260108_162240_75c3.rawdata',
#  'HeartRate_Static_zhangz_20260108_110047_c460.rawdata',
#  'MotionRecognition_allDayAcquisition_lhq_20260108_153110_75c3.rawdata',   # 异常
#  'Sleep_Nap_zhangz_20260108_165124_ead7.rawdata'


# MotionRecognition_Walk_LK_20260113_171858_c460.rawdata    电流 3 日常心率 检测到 0->1->2...->7->0 完整循环: 1 次   620秒
# MotionRecognition_allDayAcquisition_Jeven_20260112_104512_3c92.rawdata  电流 35 运动心率  没有 detection_state==2 的数据
# MotionRecognition_allDayAcquisition_Jeven_20260112_153542_3c92.rawdata  电流 42 运动心率 检测到 0->1->2...->7- >0 完整循环: 第1次: 520秒 (索引 12594-13113)
# MotionRecognition_allDayAcquisition_Jeven_20260113_101752_3c92.rawdata  电流 55 运动心率 检测到 0->1->2...->7->0 完整循环: 1 次 第1次: 228秒 9119-9347
# MotionRecognition_allDayAcquisition_Jeven_20260113_174513_3c92.rawdata  电流  7 日常心率  没有 detection_state==2 的数据 


# 十分钟办公,十分钟走路,十分钟办公 最后静止一分钟
# MotionRecognition_Walk_SHB_20260113_164744_c460       日常心率  电流 3   检测到 0->1->2...->7->0 完整循环: 1 次   609秒
# MotionRecognition_Walk_ly_20260113_163516_75c3        日常心率  电流 12   
# MotionRecognition_Walk_ljy_20260114_104751_75c3       日常心率  电流 3  
# MotionRecognition_Walk_czw_20260114_103232_c460       日常心率  电流 2     
# MotionRecognition_Walk_Cindy_20260114_153435_75c3     日常心率  电流 3
# MotionRecognition_Walk_hhx_20260114_114654_c460       日常心率  电流 2
# 

# StepCount_Daylife_Jeven_20260114_112103_3c92  日常心率  电流 2
# StepCount_Daylife_Jeven_20260114_145339_3c92  日常心率  电流 7

# StepCount_Daylife_Jeven_20260115_102128_3c92 日常心率  电流 8
# StepCount_Daylife_Jeven_20260115_030348_3c92 日常心率  电流 11



directory = r'G:\PyScript\rawdata\Motion\260119'
file_name = "MotionRecognition_Walk_hlh_20260119_185119_5a1d.rawdata"


directory = r'G:\PyScript\rawdata\Motion\260120'
file_name = "MotionRecognition_Walk_hlh_20260120_191326_5a1d.rawdata"



file_path = os.path.join(directory, file_name)
real_path = os.path.realpath(file_path)
file_name = os.path.basename(file_path)
file_name_without_ext, ext = os.path.splitext(file_name)
directory = os.path.dirname(real_path)
save_dir = os.path.join(directory, "data")
os.makedirs(save_dir, exist_ok=True)
# print(f"real_path: {real_path} \n{directory} {file_name_without_ext} {ext}")


result = get_raw_data(real_path)


data_type_dict = result.get('data_types', {})
base_info_df = result.get('base_info_df')
tyhx_df = result.get('TYHX_data_df', pd.DataFrame(columns=['unix_timestamp']))
tyhx_agc_data_df = result.get('TYHX_agc_data_df', pd.DataFrame(columns=['unix_timestamp']))

wear_info_df = result.get('wear_info_df', pd.DataFrame(columns=['unix_timestamp']))
motion_recognition_df = result.get('motion_recognition_df', pd.DataFrame(columns=['unix_timestamp']))

acc_df = result.get('acc_df')


sorted_data_types = sorted(data_type_dict.keys(), key=lambda x: int(x, 16))
readable_types = [f"{dt}  {get_full_data_type(dt)}  count: {data_type_dict[dt]} " for dt in sorted_data_types] 
print(f"文件 {file_path} 解析完成,包含数据类型: ")
print(f"{"\n".join(readable_types)}")

acc_df = acc_df[["unix_timestamp","arr_size","acc_data"]].copy() # type: ignore
df = base_info_df[['unix_timestamp','activity_level', 'heart_rate', 'step','step_freq','wear_flag',]]  # type: ignore

step_diff =  df['step'].iloc[-1] - df['step'].iloc[0]
step_diff2 = df['step'].max() - df['step'].min()
print(f"step start:{ df['step'].iloc[0]} stop:{df['step'].iloc[-1] }   diff: {step_diff }  step2: {step_diff2}   ")

print()
print(wear_info_df['ppg_g_current'].describe())

# 执行分析
# analysis_result = analyze_motion_recognition(motion_recognition_df)
# plot_motion_recognition_df(motion_recognition_df, directory, file_name_without_ext, is_show=False)
# plot_motion_recognition_df(motion_recognition_df, directory, file_name_without_ext, is_show=True)




文件 G:\PyScript\rawdata\Motion\260120\MotionRecognition_Walk_hlh_20260120_191326_5a1d.rawdata 解析完成,包含数据类型: 
00  BASE_INFO             基础信息                   count: 693 
15  RAW_PPG_GREEN         原始PPG绿光                count: 121 
17  RAW_PPG_IR            原始PPG红外                count: 679 
18  RAW_ACC               原始加速度                  count: 1388 
21  RAW_ACC_TO_PPG        原始加速度到PPG              count: 245 
33  AMBIENT_INFO          环境信息                   count: 121 
67  SLEEP_INFO            睡眠信息                   count: 693 
69  MOTION_INFO           运动识别                   count: 694 
71  WEAR_INFO             佩戴信息                   count: 688 
78  HEART_ACC_INFO        心率加速度信息                count: 121 
82  TYHX_AGC_DATA         TYHX AGC数据             count: 688 
91  GREEN_PSP_INFO        绿光PSP信息                count: 242 
96  TIME_PSP_INFO         时间PSP信息                count: 121 
99  CREEK_HR_INFO         CREEK心率信息              count: 121 
step start:2283 stop:3414   diff: 1131

In [17]:
final_merged_df = process_and_merge_dataframes(df, motion_recognition_df, acc_df)
motion_results_df = Motiondll_with_dataframe(final_merged_df, directory, file_name_without_ext)
motion_results_df = motion_results_df.rename(columns={
    'motion_type':'motion_type_ring',
    'activity_motion_time':'activity_motion_time_ring',
    'wear_flag':'wear_flag_ring'})
analysis_result = analyze_motion_recognition(motion_results_df)

日志已保存至: G:\PyScript\rawdata\Motion\260120\MotionRecognition_Walk_hlh_20260120_191326_5a1d_log.txt
运动识别数据分析报告

总运行时间: 693 秒 ( 11.55 分钟 )

数据完整性检查:
   起始状态: detection_state = 0 (正常)
   结束状态: detection_state = 4 (异常,未清空)

检测到 0->1->0 循环: 0 次


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

运动识别数据分析报告

总运行时间: 95 秒 ( 1.58 分钟 )

数据完整性检查:
   起始状态: detection_state = 0 (正常)
   结束状态: detection_state = 1 (异常,未清空)

检测到 0->1->0 循环: 0 次

检测到 0->1->2...->7->0 完整循环: 0 次

不完整循环统计: 0 次

数据结束时处于状态 1 ,循环未完成
   总时长 43 秒 ( 52-95 )
         状态 1: 43 秒 ( 53-95 )
   该段最大状态: 1



In [14]:
plot_motion_recognition_df(motion_results_df, directory, file_name_without_ext, is_show=True)

In [10]:
df = motion_recognition_df[motion_recognition_df['motion_type_ring'] ==1]

In [ ]:
# 绘制  绘制运动识别结果的多个指标
def plot_motion_recognition_results(result, directory, file_name_without_ext, is_show=False):
    """
    绘制运动识别结果的多个指标
    
    参数:
        result: DataFrame, 包含活动识别结果的数据框
        file_name_without_ext: str, 文件名(不含扩展名),用于图表标题
    """

    
    # 创建子图
    fig, axes = plt.subplots(4, 2, figsize=(18, 12), sharex=True)
    fig.suptitle(f'Activity Recognition Results {file_name_without_ext}', fontsize=16)

    # 绘制 motion_type
    axes[0, 0].plot(result.index, result['motion_type'], label='motion_type', color='blue')
    axes[0, 0].set_title('Motion Type')
    axes[0, 0].set_xlabel('Index')
    axes[0, 0].set_ylabel('Value')
    axes[0, 0].grid(True)
    axes[0, 0].legend()

    # 绘制 detection_state
    axes[0, 1].plot(result.index, result['detection_state'], label='detection_state', color='green')
    axes[0, 1].set_title('Detection State')
    axes[0, 1].set_xlabel('Index')
    axes[0, 1].set_ylabel('Value')
    axes[0, 1].grid(True)
    axes[0, 1].legend()

    # 绘制 activity_motion_time
    axes[1, 0].plot(result.index, result['activity_motion_time'], label='activity_motion_time', color='red')
    axes[1, 0].set_title('Activity Motion Time')
    axes[1, 0].set_xlabel('Index')
    axes[1, 0].set_ylabel('Time (s)')
    axes[1, 0].grid(True)
    axes[1, 0].legend()

    # 绘制 auto_detect_state
    axes[1, 1].plot(result.index, result['auto_detect_state'], label='auto_detect_state', color='purple')
    axes[1, 1].set_title('Auto Detect State')
    axes[1, 1].set_xlabel('Index')
    axes[1, 1].set_ylabel('Value')
    axes[1, 1].grid(True)
    axes[1, 1].legend()

    # 绘制 instant_type
    axes[2, 0].plot(result.index, result['instant_type'], label='instant_type', color='orange')
    axes[2, 0].set_title('Instant Type')
    axes[2, 0].set_xlabel('Index')
    axes[2, 0].set_ylabel('Value')
    axes[2, 0].grid(True)
    axes[2, 0].legend()

    # 绘制 steps_per_minute
    axes[2, 1].plot(result.index, result['steps_per_minute'], label='steps_per_minute', color='brown')
    axes[2, 1].set_title('Steps Per Minute')
    axes[2, 1].set_xlabel('Index')
    axes[2, 1].set_ylabel('Steps/min')
    axes[2, 1].grid(True)
    axes[2, 1].legend()

    # 绘制 acc_feature
    axes[3, 0].plot(result.index, result['acc_feature'], label='acc_feature', color='pink')
    axes[3, 0].set_title('ACC Feature')
    axes[3, 0].set_xlabel('Index')
    axes[3, 0].set_ylabel('Value')
    axes[3, 0].grid(True)
    axes[3, 0].legend()

    # 绘制 avg_activity_level
    axes[3, 1].plot(result.index, result['avg_activity_level'], label='avg_activity_level', color='cyan')
    axes[3, 1].set_title('Average Activity Level')
    axes[3, 1].set_xlabel('Index')
    axes[3, 1].set_ylabel('Level')
    axes[3, 1].grid(True)
    axes[3, 1].legend()

    plt.tight_layout()

    


    if is_show:
        plt.show()
    else:
        file_path = os.path.join(directory, f"{file_name_without_ext}_activity_recognition_results.png")
        plt.savefig(file_path, dpi=300)
        plt.close()
    
    # return fig




In [10]:

directory = r'G:\python\rawdata_motion\data2'

# G:\python\rawdata_motion\Typing
directory = r'G:\python\rawdata_motion\Typing'



directory = r'rawdata/Motion/251215'


# 获取目录下的所有文件
all_files = os.listdir(directory)

# 过滤出 .rawdata 文件
rawdata_files = [f for f in all_files if f.endswith('.rawdata')]

for f in rawdata_files:
    file_path = os.path.join(directory, f)

    real_path = os.path.realpath(file_path)
    file_name = os.path.basename(file_path)
    file_name_without_ext, ext = os.path.splitext(file_name)
    directory = os.path.dirname(real_path)
    info =  parse_filename(file_name)
    mac = info.get('mac','0000').lower()


    result = get_raw_data(real_path)

    data_type_dict = result.get('data_types', {})
    sorted_data_types = sorted(data_type_dict.keys(), key=lambda x: int(x, 16))
    readable_types = [f"{dt}  {get_full_data_type(dt)}  count: {data_type_dict[dt]} " for dt in sorted_data_types] 
    print(f"文件 {file_path} 解析完成,包含数据类型: ")
    print(f"{"\n".join(readable_types)}")

    motion_recognition_df = result.get('motion_recognition_df')
    base_info_df = result.get('base_info_df')
    acc_df = result.get('acc_df')
    if acc_df is None or acc_df.empty:
        print(f"文件 {file_path} 中没有加速度数据,跳过处理。")
        continue
    acc_df = acc_df[["unix_timestamp","arr_size","acc_data"]].copy() # type: ignore
    df = base_info_df[['unix_timestamp','activity_level', 'heart_rate', 'step','step_freq','wear_flag',]]  # type: ignore

    step_diff =  df['step'].iloc[-1] - df['step'].iloc[0]
    step_diff2 = df['step'].max() - df['step'].min()
    print(f"step start:{ df['step'].iloc[0]} stop:{df['step'].iloc[-1] }   diff: {step_diff }  step2: {step_diff2}   ")
    # 调用函数处理和合并 DataFrame
    final_merged_df = process_and_merge_dataframes(df, motion_recognition_df, acc_df)
    Motiondll_with_dataframe(final_merged_df, directory, file_name_without_ext)

文件 rawdata/Motion/251215\HeartRate_IndoorRun_dlh_20251216_170427_5c05.rawdata 解析完成,包含数据类型: 
00  BASE_INFO             基础信息                   count: 614 
15  RAW_PPG_GREEN         原始PPG绿光                count: 642 
17  RAW_PPG_IR            原始PPG红外                count: 642 
18  RAW_ACC               原始加速度                  count: 1284 
21  RAW_ACC_TO_PPG        原始加速度到PPG              count: 1284 
33  AMBIENT_INFO          环境信息                   count: 642 
69  MOTION_INFO           运动识别                   count: 596 
71  WEAR_INFO             佩戴信息                   count: 642 
82  TYHX_AGC_DATA         TYHX AGC数据             count: 642 
step start:7072 stop:8383   diff: 1311  step2: 1311   
日志已保存至: G:\PyScript\rawdata\Motion\251215\HeartRate_IndoorRun_dlh_20251216_170427_5c05_log.txt
文件 G:\PyScript\rawdata\Motion\251215\HeartRate_IndoorRun_dlh_20251216_170427_e61b.rawdata 解析完成,包含数据类型: 
00  BASE_INFO             基础信息                   count: 613 
15  RAW_PPG_GREEN         原始PPG绿光         

In [15]:
final_merged_df.loc[:,'acc_data'][0]

array([[ -746,   335,  -614],
       [ -764,   350,  -648],
       [ -782,   365,  -682],
       [ -785,   359,  -685],
       [ -788,   353,  -688],
       [ -777,   341,  -675],
       [ -767,   330,  -663],
       [ -771,   343,  -684],
       [ -776,   356,  -705],
       [ -781,   357,  -703],
       [ -787,   358,  -702],
       [ -776,   349,  -679],
       [ -765,   340,  -657],
       [ -760,   340,  -653],
       [ -756,   341,  -649],
       [ -756,   343,  -629],
       [ -757,   346,  -610],
       [ -722,   337,  -580],
       [ -687,   328,  -550],
       [ -682,   350,  -481],
       [ -677,   373,  -412],
       [ -660,   428,  -321],
       [ -644,   484,  -230],
       [ -643,   428,  -148],
       [ -643,   372,   -67],
       [ -719,   267,   -50],
       [ -796,   162,   -34],
       [ -935,    10,     9],
       [-1075,  -141,    52],
       [-1228,  -300,   -25],
       [-1382,  -460,  -102],
       [-1446,  -454,  -132],
       [-1511,  -448,  -162],
       [-1

In [ ]:
def save_acc_data_as_txt(final_merged_df, file_name_without_ext,dictory):
    expanded_acc_df = expand_acc_data(final_merged_df)
    json_shake_path = os.path.join(dictory, r"data/json_shake")
    data_list_path = os.path.join(dictory, r"data/creek2_step_shake_data_list.txt")
    if not os.path.exists( json_shake_path):
        os.makedirs( json_shake_path)

    acc_file_path = os.path.join(json_shake_path, file_name_without_ext + ".txt")
    data_lines = [f"{int(row['x'])} {int(row['y'])} {int(row['z'])}" 
                for _, row in expanded_acc_df[['x', 'y', 'z']].iterrows()]
    data_str = '\n'.join(data_lines)

    output = f"""header:
    {{
    waist_omron_steps: 0,
    collar_omron_steps: 0,
    reference_steps: 0,
    standard_steps: 0
    }}
    contents:
    {{
    {data_str}
    }}"""

    with open(acc_file_path, 'w') as f:
        f.write(output)



    # 将文件名写入 data/creek2_step_shake_data_list.txt
    with open(data_list_path, 'a') as f:
        f.write(file_name + '\n') 


In [ ]:
%matplotlib inline

# directory = r"G:\python\rawdata_step\计步异常数据\走路产生大量步数"
# directory = r"rawdata_step/Normal_6/Ring"
# directory = r"rawdata_step/Keyboard_clicks"
# directory = r"rawdata_step/Mouse_clicks"

directory = r"G:\python\rawdata_step\cycle"

# 获取目录下的所有文件
all_files = os.listdir(directory)

# 过滤出 .rawdata 文件
rawdata_files = [f for f in all_files if f.endswith('.rawdata')]

print(f"目录 {directory} 下的 .rawdata 文件：")
for file in rawdata_files[:]:
    file_path = os.path.join(directory,file)

    real_path = os.path.realpath(file_path)
    file_name = os.path.basename(file_path)
    file_name_without_ext, ext = os.path.splitext(file_name)
    directory = os.path.dirname(real_path)
    # print(f"real_path: {real_path} \n{directory} {file_name_without_ext} {ext}")


    result = get_raw_data(real_path)

    data_type_dict = result.get('data_types', {})
    sorted_data_types = sorted(data_type_dict.keys(), key=lambda x: int(x, 16))
    readable_types = [f"{dt}  {get_full_data_type(dt)}  count: {data_type_dict[dt]} " for dt in sorted_data_types] 
    # print(f"文件 {file_path} 解析完成,包含数据类型: ")
    # print(f"{"\n".join(readable_types)}")

    motion_recognition_df = result.get('motion_recognition_df')
    base_info_df = result.get('base_info_df')
    acc_df = result.get('acc_df')
    acc_df = acc_df[["unix_timestamp","arr_size","acc_data"]].copy() # type: ignore
    df = base_info_df[['unix_timestamp','activity_level', 'heart_rate', 'step','step_freq','wear_flag',]]  # type: ignore

    step_diff =  df['step'].iloc[-1] - df['step'].iloc[0]
    step_diff2 = df['step'].max() - df['step'].min()
    # print(f"step start:{ df['step'].iloc[0]} stop:{df['step'].iloc[-1] }   diff: {step_diff }  step2: {step_diff2}   ")
    # 调用函数处理和合并 DataFrame
    final_merged_df = process_and_merge_dataframes(df, motion_recognition_df, acc_df, directory, file_name_without_ext)
    # plot_acc_and_step_data(final_merged_df,directory, file_name_without_ext)

    step_start = final_merged_df['step'].iloc[0]
    step_end = final_merged_df['step'].iloc[-1]
    step_total = step_end - step_start
    print(f"{file_name_without_ext } step_start: {step_start}, step_end:{step_end}, total:{step_total}")
    # save_acc_data_as_txt(final_merged_df, file_name_without_ext,directory)
    plt.close()

目录 G:\python\rawdata_step\cycle 下的 .rawdata 文件：
HeartRate_OfficeWork_lk_20251210_122700_d95a step_start: 3402, step_end:3402, total:0
MotionRecognition_Walk_shb_20251210_122813_8e26 step_start: 224, step_end:224, total:0
MotionRecognition_Walk_shb_20251210_140252_8e26 step_start: 709, step_end:743, total:34


In [ ]:
# %matplotlib qt
# # directory = r"rawdata_step/计步异常数据/走路产生大量步数"
# # directory = r"rawdata_step/计步异常数据/睡眠产生大量步数"
# directory = r"G:\python\rawdata_step\cycle"


# # 获取目录下的所有文件
# all_files = os.listdir(directory)

# # 过滤出 .rawdata 文件
# rawdata_files = [f for f in all_files if f.endswith('.rawdata')]

# for file in rawdata_files:
#     file_path = os.path.join(directory,file)

#     real_path = os.path.realpath(file_path)
#     file_name = os.path.basename(file_path)
#     file_name_without_ext, ext = os.path.splitext(file_name)
#     directory = os.path.dirname(real_path)
#     # print(f"real_path: {real_path} \n{directory} {file_name_without_ext} {ext}")


#     result = get_raw_data(real_path)

#     data_type_dict = result.get('data_types', {})
#     sorted_data_types = sorted(data_type_dict.keys(), key=lambda x: int(x, 16))
#     readable_types = [f"{dt}  {get_full_data_type(dt)}  count: {data_type_dict[dt]} " for dt in sorted_data_types] 
#     # print(f"文件 {file_path} 解析完成,包含数据类型: ")
#     # print(f"{"\n".join(readable_types)}")

#     motion_recognition_df = result.get('motion_recognition_df')
#     base_info_df = result.get('base_info_df')
#     acc_df = result.get('acc_df')
#     acc_df = acc_df[["unix_timestamp","arr_size","acc_data"]].copy() # type: ignore
#     df = base_info_df[['unix_timestamp','activity_level', 'heart_rate', 'step','step_freq','wear_flag',]]  # type: ignore

#     step_diff =  df['step'].iloc[-1] - df['step'].iloc[0]
#     step_diff2 = df['step'].max() - df['step'].min()
#     # print(f"step start:{ df['step'].iloc[0]} stop:{df['step'].iloc[-1] }   diff: {step_diff }  step2: {step_diff2}   ")
#     # 调用函数处理和合并 DataFrame
#     final_merged_df = process_and_merge_dataframes(df, motion_recognition_df, acc_df, directory, file_name_without_ext)
#     plot_acc_and_step_data(final_merged_df,directory, file_name_without_ext)

#     step_start = final_merged_df['step'].iloc[0]
#     step_end = final_merged_df['step'].iloc[-1]
#     step_total = step_end - step_start
#     print(f"{ step_start }  {step_end } {step_total}")

3402  3402 0
224  224 0
709  743 34


In [ ]:
df = final_merged_df[-1500:]

In [ ]:
plot_acc_and_step_data(df,directory, file_name_without_ext)

In [ ]:
df

unix_timestamp  activity_level  heart_rate  step  step_freq  wear_flag  \
32182      1763716236               0           0  2460          0          1   
32183      1763716237               0           0  2460          0          1   
32184      1763716238               0           0  2460          0          1   
32185      1763716239               0           0  2460          0          1   
32186      1763716240               0           0  2460          0          1   
...               ...             ...         ...   ...        ...        ...   
33677      1763717731               3           0  3695          0          1   
33678      1763717732               0           0  3695          0          1   
33679      1763717733               0           0  3695          0          1   
33680      1763717734               0           0  3695          0          1   
33681      1763717735               1           0  3695          0          1   

       arr_size                                           acc_data  
32182        50  [(833, 236, -491), (831, 234, -489), (830, 233...  
32183        50  [(837, 234, -491), (835, 234, -489), (834, 235...  
32184        50  [(839, 235, -490), (837, 235, -489), (835, 235...  
32185        50  [(835, 236, -488), (834, 236, -488), (834, 237...  
32186        50  [(837, 242, -487), (834, 241, -488), (832, 240...  
...         ...                                                ...  
33677        50  [(839, 229, -490), (838, 225, -489), (837, 221...  
33678        50  [(835, 226, -494), (836, 224, -492), (837, 223...  
33679        50  [(-494, 223, -493), (173, 225, -492), (841, 22...  
33680        50  [(839, 227, -492), (837, 225, -491), (836, 224...  
33681        50  [(838, 221, -489), (838, 221, -490), (838, 222...  

[1500 rows x 8 columns]

In [ ]:
acc_df = result.get('acc_df')
acc_df

unix_timestamp  serial_number      ms            reserved  acc_type  \
0          1763684053              0  131725  [0, 0, 0, 0, 0, 0]         0   
1          1763684054              1  131224  [0, 0, 0, 0, 0, 0]         0   
2          1763684054              2  131725  [0, 0, 0, 0, 0, 0]         0   
3          1763684055              3  131225  [0, 0, 0, 0, 0, 0]         0   
4          1763684055              4  131725  [0, 0, 0, 0, 0, 0]         0   
...               ...            ...     ...                 ...       ...   
41854      1763717733            174  131756  [0, 0, 0, 0, 0, 0]         0   
41855      1763717734            175  131257  [0, 0, 0, 0, 0, 0]         0   
41856      1763717734            176  131756  [0, 0, 0, 0, 0, 0]         0   
41857      1763717735            177  131258  [0, 0, 0, 0, 0, 0]         0   
41858      1763717735            178  131757  [0, 0, 0, 0, 0, 0]         0   

       arr_bit  arr_size  freq  tolerance               data  \
0           16        50    50        100    [0, 0, 0, 0, 0]   
1           16        50    50        100    [2, 0, 0, 0, 0]   
2           16        50    50        100    [4, 0, 0, 0, 0]   
3           16        50    50        100    [6, 0, 0, 0, 0]   
4           16        50    50        100    [8, 0, 0, 0, 0]   
...        ...       ...   ...        ...                ...   
41854       16        46    50        100   [92, 0, 0, 0, 0]   
41855       16        54    50        100   [94, 0, 0, 0, 0]   
41856       16        48    50        100   [96, 0, 0, 0, 0]   
41857       16        52    50        100   [98, 0, 0, 0, 0]   
41858       16        48    50        100  [100, 0, 0, 0, 0]   

                                                acc_data  
0      [(1000, -26, 161), (1034, 16, 232), (1069, 58,...  
1      [(453, 732, 176), (381, 774, 171), (309, 816, ...  
2      [(-20, 770, 624), (-4, 767, 607), (12, 765, 59...  
3      [(525, 1101, 23), (481, 1061, 49), (438, 1021,...  
4      [(391, 918, 102), (388, 928, 104), (385, 939, ...  
...                                                  ...  
41854  [(753, 638, 202), (757, 635, 198), (761, 633, ...  
41855  [(805, 616, 185), (789, 613, 182), (774, 610, ...  
41856  [(235, 691, 261), (479, 707, 277), (724, 723, ...  
41857  [(733, 676, 242), (733, 672, 237), (734, 668, ...  
41858  [(689, 695, 424), (665, 687, 447), (641, 680, ...  

[41859 rows x 11 columns]

In [6]:
df = base_info_df[['unix_timestamp','activity_level', 'heart_rate', 'step','step_freq','wear_flag',]]  # type: ignore
df.loc[:, 'step'] = df['step'].replace(0, np.nan).interpolate(method='linear').ffill().bfill().astype(int)
step_diff =  df['step'].iloc[-1] - df['step'].iloc[0]
step_diff2 = df['step'].max() - df['step'].min()
print(f"step start:{ df['step'].iloc[0]} stop:{df['step'].iloc[-1] }   diff: {step_diff }  step2: {step_diff2}   ")
# 调用函数处理和合并 DataFrame
final_merged_df = process_and_merge_dataframes(df, motion_recognition_df, acc_df)

step start:62 stop:1155   diff: 1093  step2: 1093   


In [7]:
# 调试acc_data格式
if 'final_merged_df' in globals():
    sample_acc = final_merged_df['acc_data'].iloc[0]
    print(f"acc_data类型: {type(sample_acc)}")
    print(f"acc_data长度: {len(sample_acc) if hasattr(sample_acc, '__len__') else 'N/A'}")
    if isinstance(sample_acc, list) and len(sample_acc) > 0:
        print(f"第一个元素类型: {type(sample_acc[0])}")
        if isinstance(sample_acc[0], (list, tuple)):
            print(f"第一个元素的长度: {len(sample_acc[0])}")
            if len(sample_acc[0]) >= 3:
                print(f"样本数据: {sample_acc[0][:3]}")
            else:
                print(f"第一个元素内容: {sample_acc[0]}")

acc_data类型: <class 'numpy.ndarray'>
acc_data长度: 50


In [ ]:

import os
import sys

dll_path = r"G:\ncs_workspace\application\algorithm_lib\alg_motion_recognition_ring\build\bin\alg_motion_recognition_ring.dll"
result_list = []


try:
    with MotionRecognitionDLL(dll_path) as dll:
        # 获取版本
        version = dll.get_version()
        print(f"DLL版本: {version.major}.{version.minor}.{version.patch}")
        
        # 初始化
        dll.init_default()

        num_calls =  len(final_merged_df)
        for i in range(num_calls):
            row = final_merged_df.iloc[i]
            
            # 提取 acc_data
            acc_data = row['acc_data']
            
            # 创建 activity_data
            activity_data = MotionActivityData()
            activity_data.unix_timestamp = int(row['unix_timestamp'])
            activity_data.wear_flag = int(row['wear_flag'])
            activity_data.heartrate = int(row['heart_rate'])
            activity_data.steps = int(row['step'])
            activity_data.step_freq = int(row['step_freq'])
            activity_data.activity_level = int(row['activity_level'])
            activity_data.sport_type = 0  # 默认值
            activity_data.sport_pause_state = 0  # 默认值
            
            # 调用 process
            result = dll.process(acc_data, activity_data)
            result_list.append(result)
            # # 打印结果结构体所有字段
            # print(f"调用 {i+1}: unix_timestamp={activity_data.unix_timestamp}")
            # print(f"  Result: error_code={result.error_code}, init_timer={result.init_timer}, motion_type={result.motion_type}, detection_state={result.detection_state}")
            # print(f"  Result: activity_motion_time={result.activity_motion_time}, auto_detect_state={result.auto_detect_state}, instant_type={result.instant_type}")
            # print(f"  Result: hr_rising={result.hr_rising}, hr_avg={result.hr_avg}, steps_per_minute={result.steps_per_minute}, acc_feature={result.acc_feature}")
            # print(f"  Result: avg_activity_level={result.avg_activity_level:.2f}, wear_flag={result.wear_flag}, unworn_seconds={result.unworn_seconds}")
            # print(f"  ActivityData: unix_timestamp={activity_data.unix_timestamp}, wear_flag={activity_data.wear_flag}, heartrate={activity_data.heartrate}")
            # print(f"  ActivityData: steps={activity_data.steps}, step_freq={activity_data.step_freq}, activity_level={activity_data.activity_level}")
            # print(f"  ActivityData: sport_type={activity_data.sport_type}, sport_pause_state={activity_data.sport_pause_state}")
            # print(f"  AccData: sensor_size={len(acc_data)}, sample_count={len(acc_data)}")
            # print()

        # 将 result_list 转换为 Python DataFrame
        def result_to_dict(result):
            return {
                'error_code': result.error_code,
                'init_timer': result.init_timer,
                'motion_type': result.motion_type,
                'detection_state': result.detection_state,
                'activity_motion_time': result.activity_motion_time,
                'auto_detect_state': result.auto_detect_state,
                'instant_type': result.instant_type,
                'hr_rising': result.hr_rising,
                'hr_avg': result.hr_avg,
                'steps_per_minute': result.steps_per_minute,
                'acc_feature': result.acc_feature,
                'avg_activity_level': result.avg_activity_level,
                'wear_flag': result.wear_flag,
                'unworn_seconds': result.unworn_seconds
            }

        # 转换所有结果为字典列表
        result_dicts = [result_to_dict(r) for r in result_list]

        # 创建 DataFrame
        df_results = pd.DataFrame(result_dicts)

finally:
    pass

# 显示 DataFrame
df_results

DLL版本: 0.1.2


error_code  init_timer  motion_type  detection_state  \
0             0         648            1                2   
1             0         648            1                2   
2             0         648            1                2   
3             0         648            1                2   
4             0         648            1                2   
..          ...         ...          ...              ...   
643           0         648            1                2   
644           0         648            1                2   
645           0         648            1                2   
646           0         648            1                2   
647           0         648            1                2   

     activity_motion_time  auto_detect_state  instant_type  hr_rising  hr_avg  \
0                     287                  3             1          0      60   
1                     287                  3             1          0      60   
2                     287                  3             1          0      60   
3                     287                  3             1          0      60   
4                     287                  3             1          0      60   
..                    ...                ...           ...        ...     ...   
643                   287                  3             1          0      60   
644                   287                  3             1          0      60   
645                   287                  3             1          0      60   
646                   287                  3             1          0      60   
647                   287                  3             1          0      60   

     steps_per_minute  acc_feature  avg_activity_level  wear_flag  \
0                9071          101                 1.5          1   
1                9071          101                 1.5          1   
2                9071          101                 1.5          1   
3                9071          101                 1.5          1   
4                9071          101                 1.5          1   
..                ...          ...                 ...        ...   
643              9071          101                 1.5          1   
644              9071          101                 1.5          1   
645              9071          101                 1.5          1   
646              9071          101                 1.5          1   
647              9071          101                 1.5          1   

     unworn_seconds  
0                 0  
1                 0  
2                 0  
3                 0  
4                 0  
..              ...  
643               0  
644               0  
645               0  
646               0  
647               0  

[648 rows x 14 columns]

In [ ]:
import math

# ==========================================
# 核心算法: _fast_bearing
# ==========================================
def fast_bearing(lat1, lon1, lat2, lon2):
    """
    计算从点1到点2的方位角 (0-360度).
    使用 Flat Earth 近似公式,速度快,适合短距离导航.
    """
    deg_to_rad = math.pi / 180.0
    
    # 经度差 (X轴方向)
    d_lon = (lon2 - lon1) * deg_to_rad
    
    # 纬度修正系数 (取平均纬度的 cos 值)
    avg_lat_rad = (lat1 + lat2) / 2.0 * deg_to_rad
    scale_x = math.cos(avg_lat_rad)
    
    # 平面向量 (x, y)
    x = d_lon * scale_x
    y = (lat2 - lat1) * deg_to_rad
    
    # 计算方位角 (注意: atan2(x, y) 对应数学上的 (y, x),但在导航中通常 y 是北,x 是东)
    # math.atan2(x, y) 返回的是相对于 Y 轴(北) 的顺时针角度
    bearing_rad = math.atan2(x, y)
    bearing_deg = math.degrees(bearing_rad)
    
    # 标准化为 0-360
    return (bearing_deg + 360.0) % 360.0

def get_turn_action(angle_diff):
    """根据角度差判断动作"""
    threshold = 5.0 # 忽略 5 度以内的抖动
    if angle_diff > threshold:
        return "r"
    elif angle_diff < -threshold:
        return "l"
    else:
        return "no"

# ==========================================
# 模拟数据 (构造符合 r -> l -> r -> r 的路径)
# ==========================================
# 路径形状类似 "M" 形加上最后一段回折
points = [
    # 0. 起点
    {"lat": 22.74600, "lon": 113.92700}, 
    
    # 1. 往东北走 (Bearing ~45°) -> 在此点准备右转
    {"lat": 22.74700, "lon": 113.92800}, 
    
    # 2. 往东南走 (Bearing ~135°) -> (135 - 45 = +90 右转) -> 在此点准备左转
    {"lat": 22.74600, "lon": 113.92900}, 
    
    # 3. 往东北走 (Bearing ~45°) -> (45 - 135 = -90 左转) -> 在此点准备右转
    {"lat": 22.74700, "lon": 113.93000}, 
    
    # 4. 往东南走 (Bearing ~135°) -> (135 - 45 = +90 右转) -> 在此点准备右转(锐角)
    {"lat": 22.74600, "lon": 113.93100},
    
    # 5. 往西南走 (Bearing ~225°) -> (225 - 135 = +90 右转) -> 到达
    {"lat": 22.74500, "lon": 113.93000},
]

# ==========================================
# 主逻辑
# ==========================================
print(f"{'ID':<3} - {'Action'}")
print("-" * 10)

# 1. 先计算所有路段的方位角
segment_bearings = []
for i in range(len(points) - 1):
    b = fast_bearing(points[i]['lat'], points[i]['lon'], 
                     points[i+1]['lat'], points[i+1]['lon'])
    segment_bearings.append(b)

# 2. 遍历点,根据前后路段的角度差判断动作
for i in range(len(points)):
    action = ""
    
    if i == 0:
        action = "no" # 起点
    elif i == len(points) - 1:
        action = "arived" # 终点
    else:
        # 获取 进入该点的角度 (prev_bearing) 和 离开该点的角度 (curr_bearing)
        prev_bearing = segment_bearings[i-1] # 段 i-1
        curr_bearing = segment_bearings[i]   # 段 i
        
        # 计算角度差
        diff = curr_bearing - prev_bearing
        
        # 归一化到 -180 到 180 (复刻你的 C 代码逻辑)
        if diff > 180.0:
            diff -= 360.0
        elif diff < -180.0:
            diff += 360.0
            
        action = get_turn_action(diff)

    print(f"{i:<3} - {action}")

ID  - Action
----------
0   - no
1   - r
2   - l
3   - r
4   - r
5   - arived
